In [ ]:
%load_ext autoreload 
%autoreload 2

In [ ]:
import json
from privacypacking.utils.utils import load_logs
import pandas as pd
from experiments.ray.analysis import load_ray_experiment, load_latest_ray_experiment, load_latest_scheduling_results, load_latest_scheduling_results, load_latest_ray_experiment
import plotly.express as px
from privacypacking.budget.curves import  LaplaceCurve, GaussianCurve, SubsampledGaussianCurve
from privacypacking.budget import Budget, Task, Block
from privacypacking.schedulers.metrics import OverflowRelevance, FlatRelevance
from privacypacking.budget.block_selection import RandomBlocks
from privacypacking.utils.plot import plot_budgets
import yaml
from pathlib import Path
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import numpy as np
from experiments.ray.analysis import get_percentiles

In [ ]:
df = load_latest_scheduling_results(expname="ray/run_and_report_2022-04-06_17-27-02")

In [ ]:
df

In [ ]:
px.ecdf(
    df.query("max_blocks == 200 and data_lifetime == 1 and T == 10 and scheduler_metric == 'ArgmaxKnapsack' and allocated"),
    x="n_blocks"
)

In [ ]:
tasks_df = pd.read_csv("/home/pierre/privacypacking/data/alibaba-privacy-workload/outputs/privacy_tasks.csv")

In [ ]:
px.ecdf(
    df.query("max_blocks == 200 and data_lifetime == 1 and T == 10 and scheduler_metric == 'DominantShares' and allocated"),
    x="n_blocks"
)

Most allocated tasks have below 10 blocks, because we have T=10? The data lifetime is way too small, basically no unlocking so after T = 10 all the previous budget is unlocked and eaten up, not much space for anything else. Result: DPF does not fall into multiblock traps.

Conclusion: you should scale up everything.

In [ ]:
px.ecdf(
    df.query("max_blocks == 200 and data_lifetime == 1 and T == 10"),
    x="n_blocks"
)

In [ ]:
tasks_df.head(2)

In [ ]:
def listify(row):
    return list(map(float, row.strip('][').split(', ')))
tasks_df["alphas"] = tasks_df["alphas"].apply(listify)
tasks_df["rdp_epsilons"] = tasks_df["rdp_epsilons"].apply(listify)
tasks_df["normalized_rdp_epsilons"] = tasks_df["normalized_rdp_epsilons"].apply(listify)

In [ ]:
tasks_df["epsilon_min"] = tasks_df["normalized_rdp_epsilons"].apply(min)
tasks_df["epsilon_max"] = tasks_df["normalized_rdp_epsilons"].apply(max)

In [ ]:
tasks_df.describe()

In [ ]:
px.scatter(
    tasks_df,
    x="epsilon_max",
    y="n_blocks"
)

In [ ]:
px.scatter(
    tasks_df,
    x="epsilon_min",
    y="n_blocks"
)

In [ ]:
rdf = load_ray_experiment(Path("/home/pierre/privacypacking/logs/ray/run_and_report_2022-04-06_17-27-02"))
# rdf["scheduler_metric"] = rdf.apply(lambda row: row.scheduler_metric if row.scheduler == "basic_scheduler" else "Simplex", axis=1)
px.line(
    rdf.query("data_lifetime == 1 and max_blocks == 200").sort_values("T"),
    x="T",
    y="n_allocated_tasks",
    color="scheduler_metric",
    log_x=True,
    width=800,
    height=600,
)